In [1]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models

import matplotlib.pyplot as plt
import numpy as np


In [2]:
(train_images, train_labels), (test_images,
                               test_labels) = datasets.fashion_mnist.load_data()
train_images.shape, train_labels.shape, test_images.shape, test_labels.shape


((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [3]:
# 라벨 설정
label_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']


In [4]:
# 채널 값을 부여 : 28, 28 -> 28, 28, 1
train_images = train_images.reshape(-1, 28, 28, 1)
test_images = test_images.reshape(-1, 28, 28, 1)
train_images.shape, test_images.shape


((60000, 28, 28, 1), (10000, 28, 28, 1))

In [9]:
# AlexNet 모델 입력층에 맞게 채널 3장으로 맞춤
train_images = tf.repeat(train_images, 3, axis=3)
test_images = tf.repeat(test_images, 3, axis=3)
train_images.shape, test_images.shape


(TensorShape([60000, 28, 28, 3]), TensorShape([10000, 28, 28, 3]))

In [ ]:
# 사용할 layer module
# 논문 보면서 parameter 채우기
model = models.Sequential()
model.add(layers.Conv2D(96, (11, 11),
          activation='relu', input_shape=(227, 227, 3)))
model.add(layers.Lambda(tf.nn.local_response_normalization))
model.add(layers.MaxPooling2D((3, 3)))

model.add(layers.Conv2D(256, (5, 5), activation='relu', padding='same'))
model.add(layers.Lambda(tf.nn.local_response_normalization))
model.add(layers.MaxPooling2D((3, 3)))

model.add(layers.Conv2D(384, (3, 3), activation='relu', padding='same'))
model.add(layers.Conv2D(384, (3, 3), activation='relu', padding='same'))
model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(layers.Lambda(tf.nn.local_response_normalization))
model.add(layers.MaxPooling2D((3, 3)))  # ???

model.add(layers.Flatten())
model.add(layers.Dense(2048, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2048, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1000, activation='softmax'))


In [19]:
model = models.Sequential()

# 227 x 227 x 3으로 resize
model.add(layers.experimental.preprocessing.Resizing(227, 227,
                                                     input_shape=train_images.shape[1:]))

model.add(layers.Conv2D(96, (11, 11), activation='relu',
          input_shape=(227, 227, 3), strides=4))
model.add(layers.Lambda(tf.nn.local_response_normalization))
model.add(layers.MaxPooling2D((3, 3), strides=2))

model.add(layers.Conv2D(256, (5, 5), activation='relu', padding='same'))
model.add(layers.Lambda(tf.nn.local_response_normalization))
model.add(layers.MaxPooling2D((3, 3), strides=2))

model.add(layers.Conv2D(384, (3, 3), activation='relu', padding='same'))
model.add(layers.Conv2D(384, (3, 3), activation='relu', padding='same'))
model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(layers.Lambda(tf.nn.local_response_normalization))
model.add(layers.MaxPooling2D((3, 3), strides=2))

model.add(layers.Flatten())
model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))
# kernel 개수, 크기, stride,
# conv, pool(overlap), LRN, FC, dropout
# output shape : 1000 -> 10
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.summary()


Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resizing_6 (Resizing)       (None, 227, 227, 3)       0         
                                                                 
 conv2d_25 (Conv2D)          (None, 55, 55, 96)        34944     
                                                                 
 lambda_15 (Lambda)          (None, 55, 55, 96)        0         
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 27, 27, 96)       0         
 g2D)                                                            
                                                                 
 conv2d_26 (Conv2D)          (None, 27, 27, 256)       614656    
                                                                 
 lambda_16 (Lambda)          (None, 27, 27, 256)       0         
                                                      

In [20]:
model.fit(train_images, train_labels, batch_size=128,
          epochs=10, validation_split=0.2)


Epoch 1/10
375/375 [==============================] - 685s 2s/step - loss: 0.6412 - accuracy: 0.7638 - val_loss: 0.3398 - val_accuracy: 0.8717
Epoch 2/10
375/375 [==============================] - 686s 2s/step - loss: 0.3468 - accuracy: 0.8716 - val_loss: 0.3124 - val_accuracy: 0.8821
Epoch 3/10
375/375 [==============================] - 681s 2s/step - loss: 0.3225 - accuracy: 0.8802 - val_loss: 0.3012 - val_accuracy: 0.8865
Epoch 4/10
375/375 [==============================] - 679s 2s/step - loss: 0.2998 - accuracy: 0.8879 - val_loss: 0.2881 - val_accuracy: 0.8923
Epoch 5/10
375/375 [==============================] - 676s 2s/step - loss: 0.2836 - accuracy: 0.8930 - val_loss: 0.2888 - val_accuracy: 0.8892
Epoch 6/10
375/375 [==============================] - 689s 2s/step - loss: 0.2688 - accuracy: 0.8993 - val_loss: 0.2717 - val_accuracy: 0.8997
Epoch 7/10
375/375 [==============================] - 682s 2s/step - loss: 0.2527 - accuracy: 0.9061 - val_loss: 0.2552 - val_accuracy: 0.9046

In [22]:
model.evaluate(test_images, test_labels)


313/313 [==============================] - 61s 196ms/step - loss: 0.2803 - accuracy: 0.9043


[0.28028249740600586, 0.9042999744415283]

In [23]:
model.save('my_model.h5')
# new_model = keras.models.load_model('my_model.h5')


In [ ]:
# AlexNet 모델을 간소화해봅시다
# resize없이 (28, 28, 3)으로 받아봅시다

# ** activation 은 모두 relu로 유지
# layer 1 : 32개, (3, 3), strides=1 / pooling은 (2, 2) overlapping x / normalizing 유지

# layer 2 : 64개, (3, 3), strides=1 / pooling은 (2, 2) overlapping x / normalizing 유지

# layer 3 : 128개, (3, 3), strides=1 / pooling은 (2, 2) overlapping x

# layer 4, 5 : 삭제

# fully connected layer 1, 2 : node 원하는대로 주기
